   # <center> New York City public school project

<center>Data was taken from dataquest.io -> https://data.cityofnewyork.us

One of the most controversial issues in the U.S. educational system is the efficacy of standardized tests and whether they're unfair to certain groups. **Investigating the correlations between SAT scores and demographics might be an interesting angle to take.** We could correlate SAT scores with factors like race, gender, income, and more.

The SAT, or Scholastic Aptitude Test, is an exam that U.S. high school students take before applying to college. Colleges take the test scores into account when deciding who to admit, so it's important to perform well.

The test consists of three sections, each of which has 800 possible points. The combined score is out of 2,400 possible points (while this number has changed a few times, the dataset for this project is based on 2,400 total points). Organizations often rank high schools by their average SAT scores. The scores are also considered a measure of overall school district quality.

### Preliminary important information
-  Only high school students take the SAT, so we'll want to focus on high schools. <br>
-  New York City is made up of five boroughs, which are essentially distinct regions. <br>
-  New York City schools fall within several different school districts, each of which can contain dozens of schools. <br>
-  Our datasets include several different types of schools. We'll need to clean them so that we can focus on high schools only. <br>
-  Each school in New York City has a unique code called a DBN or district borough number.<br>
-  Aggregating data by district allows us to use the district mapping data to plot district-by-district differences.<br>

In [36]:
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
import re
from IPython.display import display, HTML

In [37]:
# let's add each of the dataframes to the dictionary data, using the base of the filename as the key and the corresponding dataframe as its value

data_files = [
    "ap_2010.csv",
    "class_size.csv",
    "demographics.csv",
    "graduation.csv",
    "hs_directory.csv",
    "sat_results.csv"
]

data = {}

for file in data_files:
    data.update({file.split('.csv')[0] : pd.read_csv('{}'.format(file))})

In [38]:
# explored sat_results to see what we can discover

data['sat_results'].head()

,DBN,SCHOOL NAME,Num of SAT Test Takers,SAT Critical Reading Avg. Score,SAT Math Avg. Score,SAT Writing Avg. Score
0,01M292,HENRY STREET SCHOOL FOR INTERNATIONAL STUDIES,29,355,404,363
1,01M448,UNIVERSITY NEIGHBORHOOD HIGH SCHOOL,91,383,423,366
2,01M450,EAST SIDE COMMUNITY SCHOOL,70,377,402,370
3,01M458,FORSYTH SATELLITE ACADEMY,7,414,401,359
4,01M509,MARTA VALLE HIGH SCHOOL,44,390,433,384


-  The DBN appears to be a unique ID for each school.
-  We can tell from the first few rows of names that we only have data about high schools.
-  There's only a single row for each high school, so each DBN is unique in the SAT data.
-  We may eventually want to combine the three columns that contain SAT scores: SAT Critical Reading Avg. Score, SAT Math Avg. Score, and SAT Writing Avg. Score into a single column to make the scores easier to analyze.

Given these observations, let's explore the other datasets to see if we can gain any insight into how to combine them.

In [39]:
for key in data:
    display(data[key].head(3))

,DBN,SchoolName,AP Test Takers,Total Exams Taken,Number of Exams with scores 3 4 or 5
0,01M448,UNIVERSITY NEIGHBORHOOD H.S.,39.0,49.0,10.0
1,01M450,EAST SIDE COMMUNITY HS,19.0,21.0,NaN
2,01M515,LOWER EASTSIDE PREP,24.0,26.0,24.0


,CSD,BOROUGH,SCHOOL CODE,SCHOOL NAME,GRADE,PROGRAM TYPE,CORE SUBJECT (MS CORE and 9-12 ONLY),CORE COURSE (MS CORE and 9-12 ONLY),SERVICE CATEGORY(K-9* ONLY),NUMBER OF STUDENTS / SEATS FILLED,NUMBER OF SECTIONS,AVERAGE CLASS SIZE,SIZE OF SMALLEST CLASS,SIZE OF LARGEST CLASS,DATA SOURCE,SCHOOLWIDE PUPIL-TEACHER RATIO
0,1,M,M015,P.S. 015 Roberto Clemente,0K,GEN ED,-,-,-,19.0,1.0,19.0,19.0,19.0,ATS,NaN
1,1,M,M015,P.S. 015 Roberto Clemente,0K,CTT,-,-,-,21.0,1.0,21.0,21.0,21.0,ATS,NaN
2,1,M,M015,P.S. 015 Roberto Clemente,01,GEN ED,-,-,-,17.0,1.0,17.0,17.0,17.0,ATS,NaN


,DBN,Name,schoolyear,fl_percent,frl_percent,total_enrollment,prek,k,grade1,grade2,...,black_num,black_per,hispanic_num,hispanic_per,white_num,white_per,male_num,male_per,female_num,female_per
0,01M015,P.S. 015 ROBERTO CLEMENTE,20052006,89.4,NaN,281,15,36,40,33,...,74,26.3,189,67.3,5,1.8,158.0,56.2,123.0,43.8
1,01M015,P.S. 015 ROBERTO CLEMENTE,20062007,89.4,NaN,243,15,29,39,38,...,68,28.0,153,63.0,4,1.6,140.0,57.6,103.0,42.4
2,01M015,P.S. 015 ROBERTO CLEMENTE,20072008,89.4,NaN,261,18,43,39,36,...,77,29.5,157,60.2,7,2.7,143.0,54.8,118.0,45.2


,Demographic,DBN,School Name,Cohort,Total Cohort,Total Grads - n,Total Grads - % of cohort,Total Regents - n,Total Regents - % of cohort,Total Regents - % of grads,...,Regents w/o Advanced - n,Regents w/o Advanced - % of cohort,Regents w/o Advanced - % of grads,Local - n,Local - % of cohort,Local - % of grads,Still Enrolled - n,Still Enrolled - % of cohort,Dropped Out - n,Dropped Out - % of cohort
0,Total Cohort,01M292,HENRY STREET SCHOOL FOR INTERNATIONAL,2003,5,s,NaN,s,NaN,NaN,...,s,NaN,NaN,s,NaN,NaN,s,NaN,s,NaN
1,Total Cohort,01M292,HENRY STREET SCHOOL FOR INTERNATIONAL,2004,55,37,67.3,17,30.9,45.9,...,17,30.9,45.9,20,36.4,54.1,15,27.3,3,5.5
2,Total Cohort,01M292,HENRY STREET SCHOOL FOR INTERNATIONAL,2005,64,43,67.2,27,42.2,62.8,...,27,42.2,62.8,16,25.0,37.2,9,14.1,9,14.1


,dbn,school_name,borough,building_code,phone_number,fax_number,grade_span_min,grade_span_max,expgrade_span_min,expgrade_span_max,...,priority08,priority09,priority10,Location 1,Community Board,Council District,Census Tract,BIN,BBL,NTA
0,27Q260,Frederick Douglass Academy VI High School,Queens,Q465,718-471-2154,718-471-2890,9.0,12,NaN,NaN,...,NaN,NaN,NaN,"8 21 Bay 25 Street\nFar Rockaway, NY 11691\n(4...",14.0,31.0,100802.0,4300730.0,4.157360e+09,Far Rockaway-Bayswater ...
1,21K559,Life Academy High School for Film and Music,Brooklyn,K400,718-333-7750,718-333-7775,9.0,12,NaN,NaN,...,NaN,NaN,NaN,"2630 Benson Avenue\nBrooklyn, NY 11214\n(40.59...",13.0,47.0,306.0,3186454.0,3.068830e+09,Gravesend ...
2,16K393,Frederick Douglass Academy IV Secondary School,Brooklyn,K026,718-574-2820,718-574-2821,9.0,12,NaN,NaN,...,NaN,NaN,NaN,"1014 Lafayette Avenue\nBrooklyn, NY 11221\n(40...",3.0,36.0,291.0,3393805.0,3.016160e+09,Stuyvesant Heights ...


,DBN,SCHOOL NAME,Num of SAT Test Takers,SAT Critical Reading Avg. Score,SAT Math Avg. Score,SAT Writing Avg. Score
0,01M292,HENRY STREET SCHOOL FOR INTERNATIONAL STUDIES,29,355,404,363
1,01M448,UNIVERSITY NEIGHBORHOOD HIGH SCHOOL,91,383,423,366
2,01M450,EAST SIDE COMMUNITY SCHOOL,70,377,402,370


In [40]:
# let's take a closer look at the second dataframe

data['class_size'].head(3)

,CSD,BOROUGH,SCHOOL CODE,SCHOOL NAME,GRADE,PROGRAM TYPE,CORE SUBJECT (MS CORE and 9-12 ONLY),CORE COURSE (MS CORE and 9-12 ONLY),SERVICE CATEGORY(K-9* ONLY),NUMBER OF STUDENTS / SEATS FILLED,NUMBER OF SECTIONS,AVERAGE CLASS SIZE,SIZE OF SMALLEST CLASS,SIZE OF LARGEST CLASS,DATA SOURCE,SCHOOLWIDE PUPIL-TEACHER RATIO
0,1,M,M015,P.S. 015 Roberto Clemente,0K,GEN ED,-,-,-,19.0,1.0,19.0,19.0,19.0,ATS,NaN
1,1,M,M015,P.S. 015 Roberto Clemente,0K,CTT,-,-,-,21.0,1.0,21.0,21.0,21.0,ATS,NaN
2,1,M,M015,P.S. 015 Roberto Clemente,01,GEN ED,-,-,-,17.0,1.0,17.0,17.0,17.0,ATS,NaN


We can make some observations based on the first few rows of the dataframes:

-  Each dataset appears to either have a DBN column or the information we need to create one. That means we can use a DBN column to combine the datasets. First we'll pinpoint matching rows from different datasets by looking for identical DBNs, then group all of their columns together in a single dataset.
- Some fields look interesting for mapping -- particularly Location 1, which contains coordinates inside a larger string.
- Some of the datasets appear to contain multiple rows for each school (because the rows have duplicate DBN values). That means we’ll have to do some preprocessing to ensure that each DBN is unique within each dataset. If we don't do this, we'll run into problems when we combine the datasets, because we might be merging two rows in one data set with one row in another dataset.

In [41]:
# downloaded the surveys that will also be needed

survey_d75 = pd.read_csv('survey_d75.txt', sep='\t', encoding='Windows-1252')
survey_all = pd.read_csv('survey_all.txt', sep='\t', encoding='Windows-1252')

# combined it into a single dataframe

survey = pd.concat([survey_d75, survey_all], axis=0)
survey.head(5)

,dbn,bn,schoolname,d75,studentssurveyed,highschool,schooltype,rr_s,rr_t,rr_p,...,s_N_q14e_3,s_N_q14e_4,s_N_q14f_1,s_N_q14f_2,s_N_q14f_3,s_N_q14f_4,s_N_q14g_1,s_N_q14g_2,s_N_q14g_3,s_N_q14g_4
0,75K004,K004,P.S. K004,1,Yes,0.0,District 75 Special Education,38.0,90,72,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,75K036,K036,P.S. 36,1,Yes,NaN,District 75 Special Education,70.0,69,44,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,75K053,K053,P.S. K053,1,Yes,NaN,District 75 Special Education,94.0,97,53,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,75K077,K077,P.S. K077,1,Yes,NaN,District 75 Special Education,95.0,65,55,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,75K140,K140,P.S. K140,1,Yes,0.0,District 75 Special Education,77.0,70,42,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


There are two immediate facts that we can see in the data:

- There are over 2000 columns, nearly all of which we don't need. We'll have to filter the data to remove the unnecessary ones. Working with fewer columns makes it easier to print the dataframe out and find correlations within it.
- The survey data has a dbn column that we'll want to convert to uppercase (DBN). The conversion makes the column name consistent with the other data sets.

First, we'll need to filter the columns to remove the ones we don't need. Luckily, there's a data dictionary at the original data download location. The dictionary tells us what each column represents. Based on our knowledge of the problem and the analysis we're trying to do, we can use the data dictionary to determine which columns to use.

Based on the dictionary(https://data.cityofnewyork.us/Education/2011-NYC-School-Survey/mnz3-dyi8), it looks like these are the relevant columns:

["dbn", "rr_s", "rr_t", "rr_p", "N_s", "N_t", "N_p", "saf_p_11", "com_p_11", "eng_p_11", "aca_p_11", "saf_t_11", "com_t_11", "eng_t_11", "aca_t_11", "saf_s_11", "com_s_11", "eng_s_11", "aca_s_11", "saf_tot_11", "com_tot_11", "eng_tot_11", "aca_tot_11"]

Before we filter columns out, we'll want to copy the data from the dbn column into a new column called DBN.

In [42]:
# let's change the 'dbn' column name to uppercase

survey['DBN'] = survey['dbn']

# reassigned the variable with only the necessary columns

survey = survey[["DBN", "rr_s", "rr_t", "rr_p", "N_s", "N_t", "N_p", "saf_p_11", "com_p_11", "eng_p_11", "aca_p_11", "saf_t_11", "com_t_11", "eng_t_11", "aca_t_11", "saf_s_11", "com_s_11", "eng_s_11", "aca_s_11", "saf_tot_11", "com_tot_11", "eng_tot_11", "aca_tot_11"]]

# added the survey dataframe to the distionary data

data['survey'] = survey

C:\Users\nlero\AppData\Local\Temp\ipykernel_19412\4233832798.py:3: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`



In [43]:
# let's create "DBN" column in the class size dataframe by checking the "CSD" column on length 
# if it is less than two digits long add a leading 0. Then add the "SCHOOL CODE" column

data['class_size']['DBN'] = data['class_size']['CSD'].apply(lambda x: str(x) if len(str(x)) > 1 else '0{}'.format(str(x))) \
                          + data['class_size']['SCHOOL CODE']

In [44]:
data['class_size']['DBN']

0        01M015
1        01M015
2        01M015
3        01M015
4        01M015
          ...  
27606    32K564
27607    32K564
27608    32K564
27609    32K564
27610    32K564
Name: DBN, Length: 27611, dtype: object

In [45]:
# let's create a column that totals up the SAT scores for the different sections of the exam. 
# This makes it much easier to correlate scores with demographic factors because we'll be working with a single number, 
# rather than three different ones.

data['sat_results']['sat_score'] = pd.to_numeric(data['sat_results']['SAT Math Avg. Score'],errors="coerce")\
                                 + pd.to_numeric(data['sat_results']['SAT Critical Reading Avg. Score'],errors="coerce")\
                                 + pd.to_numeric(data['sat_results']['SAT Writing Avg. Score'], errors="coerce")

In [46]:
# let's extract the latitude using regular expressions

data['hs_directory']['lat'] = data['hs_directory']['Location 1']\
                              .apply(lambda x: re.findall(r'([.\d-]+),\s([.\d-]+)',str(x))[0][0])

data['hs_directory']['lat']

0      40.601989336
1      40.593593811
2      40.692133704
3      40.822303765
4      40.773670507
           ...     
430    40.765026708
431    40.840373398
432    40.696961848
433    40.732485378
434     40.58211615
Name: lat, Length: 435, dtype: object

In [47]:
# then do the same with longitude

data['hs_directory']['lon'] = data['hs_directory']['Location 1']\
                              .apply(lambda x: re.findall(r'([.\d-]+),\s([.\d-]+)',str(x))[0][1])

data['hs_directory']['lon']

0      -73.762834323
1      -73.984729232
2      -73.931503172
3       -73.85596139
4      -73.985268558
           ...      
430    -73.992516578
431    -73.910838008
432    -73.910815939
433    -73.983053388
434    -73.972894492
Name: lon, Length: 435, dtype: object

In [48]:
# changed the lat and the lon columns to numeric

data['hs_directory']['lat'] = pd.to_numeric(data['hs_directory']['lat'],errors="coerce")
data['hs_directory']['lon'] = pd.to_numeric(data['hs_directory']['lon'],errors="coerce")

In [49]:
data['hs_directory'].rename(columns={'dbn':'DBN'}, inplace=True)

We noticed that the values in the DBN column were unique in the sat_results data set. Other data sets like class_size had duplicate DBN values.
We'll need to condense these datasets so that each value in the DBN column is unique.
To resolve this issue, we'll condense the class_size, graduation and demographics datasets so that each DBN is unique.
The first dataset that we'll condense is class_size.

In [50]:
data['class_size'].head()

,CSD,BOROUGH,SCHOOL CODE,SCHOOL NAME,GRADE,PROGRAM TYPE,CORE SUBJECT (MS CORE and 9-12 ONLY),CORE COURSE (MS CORE and 9-12 ONLY),SERVICE CATEGORY(K-9* ONLY),NUMBER OF STUDENTS / SEATS FILLED,NUMBER OF SECTIONS,AVERAGE CLASS SIZE,SIZE OF SMALLEST CLASS,SIZE OF LARGEST CLASS,DATA SOURCE,SCHOOLWIDE PUPIL-TEACHER RATIO,DBN
0,1,M,M015,P.S. 015 Roberto Clemente,0K,GEN ED,-,-,-,19.0,1.0,19.0,19.0,19.0,ATS,NaN,01M015
1,1,M,M015,P.S. 015 Roberto Clemente,0K,CTT,-,-,-,21.0,1.0,21.0,21.0,21.0,ATS,NaN,01M015
2,1,M,M015,P.S. 015 Roberto Clemente,01,GEN ED,-,-,-,17.0,1.0,17.0,17.0,17.0,ATS,NaN,01M015
3,1,M,M015,P.S. 015 Roberto Clemente,01,CTT,-,-,-,17.0,1.0,17.0,17.0,17.0,ATS,NaN,01M015
4,1,M,M015,P.S. 015 Roberto Clemente,02,GEN ED,-,-,-,15.0,1.0,15.0,15.0,15.0,ATS,NaN,01M015


The first few rows all pertain to the same school, which is why the DBN appears more than once. It looks like each school has multiple values for GRADE, PROGRAM TYPE, CORE SUBJECT (MS CORE and 9-12 ONLY), and CORE COURSE (MS CORE and 9-12 ONLY).

If we look at the unique values for GRADE, we get the following:

In [51]:
data['class_size'] = data['class_size'].rename(columns={'GRADE ':'GRADE'})
data['class_size']['GRADE'].unique()

array(['0K', '01', '02', '03', '04', '05', '0K-09', nan, '06', '07', '08',
       'MS Core', '09-12', '09'], dtype=object)

Since we're dealing with high schools, we're only concerned with grades 9 through 12. That means we only want to pick rows where the value in the GRADE column is 09-12.

If we look at the unique values for PROGRAM TYPE, we get the following:

In [52]:
data['class_size']['PROGRAM TYPE'].unique()

array(['GEN ED', 'CTT', 'SPEC ED', nan, 'G&T'], dtype=object)

Each school can have multiple program types. Since GEN ED is the largest category by far, let's only select rows where PROGRAM TYPE is GEN ED.

In [59]:
class_size = data['class_size']
class_size = class_size.query('GRADE  == "09-12" or GRADE  == "09"')              
class_size = class_size.query('`PROGRAM TYPE` == "GEN ED"')
class_size.head()

,CSD,BOROUGH,SCHOOL CODE,SCHOOL NAME,GRADE,PROGRAM TYPE,CORE SUBJECT (MS CORE and 9-12 ONLY),CORE COURSE (MS CORE and 9-12 ONLY),SERVICE CATEGORY(K-9* ONLY),NUMBER OF STUDENTS / SEATS FILLED,NUMBER OF SECTIONS,AVERAGE CLASS SIZE,SIZE OF SMALLEST CLASS,SIZE OF LARGEST CLASS,DATA SOURCE,SCHOOLWIDE PUPIL-TEACHER RATIO,DBN
225,1,M,M292,Henry Street School for International Studies,09-12,GEN ED,ENGLISH,English 9,-,63.0,3.0,21.0,19.0,25.0,STARS,NaN,01M292
226,1,M,M292,Henry Street School for International Studies,09-12,GEN ED,ENGLISH,English 10,-,79.0,3.0,26.3,24.0,31.0,STARS,NaN,01M292
227,1,M,M292,Henry Street School for International Studies,09-12,GEN ED,ENGLISH,English 11,-,38.0,2.0,19.0,16.0,22.0,STARS,NaN,01M292
228,1,M,M292,Henry Street School for International Studies,09-12,GEN ED,ENGLISH,English 12,-,69.0,3.0,23.0,13.0,30.0,STARS,NaN,01M292
229,1,M,M292,Henry Street School for International Studies,09-12,GEN ED,MATH,Integrated Algebra,-,53.0,3.0,17.7,16.0,21.0,STARS,NaN,01M292


As we saw when we displayed class_size on the last screen, DBN still isn't completely unique. This is due to the CORE COURSE (MS CORE and 9-12 ONLY) and CORE SUBJECT (MS CORE and 9-12 ONLY) columns.

CORE COURSE (MS CORE and 9-12 ONLY) and CORE SUBJECT (MS CORE and 9-12 ONLY) seem to pertain to different kinds of classes. For example, here are the unique values for CORE SUBJECT (MS CORE and 9-12 ONLY):

In [60]:
class_size['CORE SUBJECT (MS CORE and 9-12 ONLY)'].unique()

array(['ENGLISH', 'MATH', 'SCIENCE', 'SOCIAL STUDIES', '-'], dtype=object)

This column only seems to include certain subjects. We want our class size data to include every single class a school offers -- not just a subset of them. What we can do is take the average across all of the classes a school offers. This gives us unique DBN values, while also incorporating as much data as possible into the average.

In [61]:
class_size = class_size.groupby('DBN', as_index=False).agg(np.mean)
data['class_size'] = class_size
class_size.head()

,DBN,CSD,NUMBER OF STUDENTS / SEATS FILLED,NUMBER OF SECTIONS,AVERAGE CLASS SIZE,SIZE OF SMALLEST CLASS,SIZE OF LARGEST CLASS,SCHOOLWIDE PUPIL-TEACHER RATIO
0,01M292,1.0,88.0000,4.000000,22.564286,18.50,26.571429,NaN
1,01M332,1.0,46.0000,2.000000,22.000000,21.00,23.500000,NaN
2,01M378,1.0,33.0000,1.000000,33.000000,33.00,33.000000,NaN
3,01M448,1.0,105.6875,4.750000,22.231250,18.25,27.062500,NaN
4,01M450,1.0,57.6000,2.733333,21.200000,19.40,22.866667,NaN


Let's condense demographics. The first few rows look like this:

In [62]:
data['demographics'].head(3)

,DBN,Name,schoolyear,fl_percent,frl_percent,total_enrollment,prek,k,grade1,grade2,...,black_num,black_per,hispanic_num,hispanic_per,white_num,white_per,male_num,male_per,female_num,female_per
0,01M015,P.S. 015 ROBERTO CLEMENTE,20052006,89.4,NaN,281,15,36,40,33,...,74,26.3,189,67.3,5,1.8,158.0,56.2,123.0,43.8
1,01M015,P.S. 015 ROBERTO CLEMENTE,20062007,89.4,NaN,243,15,29,39,38,...,68,28.0,153,63.0,4,1.6,140.0,57.6,103.0,42.4
2,01M015,P.S. 015 ROBERTO CLEMENTE,20072008,89.4,NaN,261,18,43,39,36,...,77,29.5,157,60.2,7,2.7,143.0,54.8,118.0,45.2


In this case, the only column that prevents a given DBN from being unique is schoolyear. We only want to select rows where schoolyear is 20112012. This will give us the most recent year of data, and also match our SAT results data.

In [63]:
data['demographics'] = data['demographics'].query('schoolyear == 20112012')
data['demographics'].head(3)

,DBN,Name,schoolyear,fl_percent,frl_percent,total_enrollment,prek,k,grade1,grade2,...,black_num,black_per,hispanic_num,hispanic_per,white_num,white_per,male_num,male_per,female_num,female_per
6,01M015,P.S. 015 ROBERTO CLEMENTE,20112012,NaN,89.4,189,13,31,35,28,...,63,33.3,109,57.7,4,2.1,97.0,51.3,92.0,48.7
13,01M019,P.S. 019 ASHER LEVY,20112012,NaN,61.5,328,32,46,52,54,...,81,24.7,158,48.2,28,8.5,147.0,44.8,181.0,55.2
20,01M020,PS 020 ANNA SILVER,20112012,NaN,92.5,626,52,102,121,87,...,55,8.8,357,57.0,16,2.6,330.0,52.7,296.0,47.3


Finally, we'll need to condense the graduation dataset.

The Demographic and Cohort columns are what prevent DBN from being unique in the graduation data. A Cohort appears to refer to the year the data represents, and the Demographic appears to refer to a specific demographic group. In this case, we want to pick data from the most recent Cohort available, which is 2006. We also want data from the full cohort, so we'll only pick rows where Demographic is Total Cohort.

In [64]:
data['graduation'] = data['graduation'].query('Cohort == "2006" | Cohort == "2006 Aug"')
data['graduation'] = data['graduation'].query('Demographic == "Total Cohort"')
data['graduation'].head(3)

,Demographic,DBN,School Name,Cohort,Total Cohort,Total Grads - n,Total Grads - % of cohort,Total Regents - n,Total Regents - % of cohort,Total Regents - % of grads,...,Regents w/o Advanced - n,Regents w/o Advanced - % of cohort,Regents w/o Advanced - % of grads,Local - n,Local - % of cohort,Local - % of grads,Still Enrolled - n,Still Enrolled - % of cohort,Dropped Out - n,Dropped Out - % of cohort
3,Total Cohort,01M292,HENRY STREET SCHOOL FOR INTERNATIONAL,2006,78,43,55.1,36,46.2,83.7,...,36,46.2,83.7,7,9.0,16.3,16,20.5,11,14.1
4,Total Cohort,01M292,HENRY STREET SCHOOL FOR INTERNATIONAL,2006 Aug,78,44,56.4,37,47.4,84.1,...,37,47.4,84.1,7,9.0,15.9,15,19.2,11,14.1
10,Total Cohort,01M448,UNIVERSITY NEIGHBORHOOD HIGH SCHOOL,2006,124,53,42.7,42,33.9,79.2,...,34,27.4,64.2,11,8.9,20.8,46,37.1,20,16.1


In [65]:
data['graduation']['Cohort'].replace({'2006 Aug':'2006'}, inplace=True)
data['graduation'].head(3)

,Demographic,DBN,School Name,Cohort,Total Cohort,Total Grads - n,Total Grads - % of cohort,Total Regents - n,Total Regents - % of cohort,Total Regents - % of grads,...,Regents w/o Advanced - n,Regents w/o Advanced - % of cohort,Regents w/o Advanced - % of grads,Local - n,Local - % of cohort,Local - % of grads,Still Enrolled - n,Still Enrolled - % of cohort,Dropped Out - n,Dropped Out - % of cohort
3,Total Cohort,01M292,HENRY STREET SCHOOL FOR INTERNATIONAL,2006,78,43,55.1,36,46.2,83.7,...,36,46.2,83.7,7,9.0,16.3,16,20.5,11,14.1
4,Total Cohort,01M292,HENRY STREET SCHOOL FOR INTERNATIONAL,2006,78,44,56.4,37,47.4,84.1,...,37,47.4,84.1,7,9.0,15.9,15,19.2,11,14.1
10,Total Cohort,01M448,UNIVERSITY NEIGHBORHOOD HIGH SCHOOL,2006,124,53,42.7,42,33.9,79.2,...,34,27.4,64.2,11,8.9,20.8,46,37.1,20,16.1


In [66]:
data['graduation'] = data['graduation'].groupby('DBN', as_index=False).agg(np.mean)
data['graduation'].head(3)

,DBN,Total Cohort,Total Grads - % of cohort,Total Regents - % of cohort,Total Regents - % of grads,Advanced Regents - % of cohort,Advanced Regents - % of grads,Regents w/o Advanced - % of cohort,Regents w/o Advanced - % of grads,Local - % of cohort,Local - % of grads,Still Enrolled - % of cohort,Dropped Out - % of cohort
0,01M292,78.0,55.75,46.8,83.90,0.0,0.0,46.8,83.90,9.0,16.10,19.85,14.1
1,01M448,124.0,45.55,33.9,74.60,6.5,14.2,27.4,60.45,11.7,25.40,34.30,16.1
2,01M450,90.0,78.35,75.0,95.75,0.0,0.0,75.0,95.75,3.3,4.25,16.15,5.6


#### AP exams have a 1 to 5 scale; 3 or higher is a passing score. Many high school students take AP exams -- particularly those who attend academically challenging institutions. AP exams are rarer in schools that lack funding or academic rigor.

It will be interesting to find out whether AP exam scores are correlated with SAT scores across high schools. To determine this, we'll need to convert the AP exam scores in the ap_2010 data set to numeric values first.

In [67]:
cols = ['AP Test Takers ', 'Total Exams Taken', 'Number of Exams with scores 3 4 or 5']
for col in cols:
    data["ap_2010"][col] = pd.to_numeric(data["ap_2010"][col], errors="coerce")

In [68]:
combined = data["sat_results"]

combined = combined.merge(data['ap_2010'],how='left', on='DBN')
combined = combined.merge(data['graduation'],how='left', on='DBN')

In [69]:
combined.shape

(479, 23)

In [70]:
combined.head(2)

,DBN,SCHOOL NAME,Num of SAT Test Takers,SAT Critical Reading Avg. Score,SAT Math Avg. Score,SAT Writing Avg. Score,sat_score,SchoolName,AP Test Takers,Total Exams Taken,...,Total Regents - % of cohort,Total Regents - % of grads,Advanced Regents - % of cohort,Advanced Regents - % of grads,Regents w/o Advanced - % of cohort,Regents w/o Advanced - % of grads,Local - % of cohort,Local - % of grads,Still Enrolled - % of cohort,Dropped Out - % of cohort
0,01M292,HENRY STREET SCHOOL FOR INTERNATIONAL STUDIES,29,355,404,363,1122.0,NaN,NaN,NaN,...,46.8,83.9,0.0,0.0,46.8,83.90,9.0,16.1,19.85,14.1
1,01M448,UNIVERSITY NEIGHBORHOOD HIGH SCHOOL,91,383,423,366,1172.0,UNIVERSITY NEIGHBORHOOD H.S.,39.0,49.0,...,33.9,74.6,6.5,14.2,27.4,60.45,11.7,25.4,34.30,16.1


In [71]:
to_merge = ["class_size", "demographics", "survey", "hs_directory"]

for m in to_merge:
    combined = combined.merge(data[m], on="DBN", how="inner")
combined.shape


(364, 154)

In [72]:
combined = combined.fillna(combined.mean())
combined = combined.fillna(0)

C:\Users\nlero\AppData\Local\Temp\ipykernel_19412\3616125232.py:1: FutureWarning:

Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.



In [73]:
combined.head()

,DBN,SCHOOL NAME,Num of SAT Test Takers,SAT Critical Reading Avg. Score,SAT Math Avg. Score,SAT Writing Avg. Score,sat_score,SchoolName,AP Test Takers,Total Exams Taken,...,priority10,Location 1,Community Board,Council District,Census Tract,BIN,BBL,NTA,lat,lon
0,01M292,HENRY STREET SCHOOL FOR INTERNATIONAL STUDIES,29,355,404,363,1122.0,0,129.028846,197.038462,...,0.0,"220 Henry Street\nNew York, NY 10002\n(40.7137...",3.0,1.0,201.0,1003223.0,1.002690e+09,Lower East Side ...,40.713764,-73.985260
1,01M448,UNIVERSITY NEIGHBORHOOD HIGH SCHOOL,91,383,423,366,1172.0,UNIVERSITY NEIGHBORHOOD H.S.,39.000000,49.000000,...,0.0,"200 Monroe Street\nNew York, NY 10002\n(40.712...",3.0,1.0,202.0,1003214.0,1.002590e+09,Lower East Side ...,40.712332,-73.984797
2,01M450,EAST SIDE COMMUNITY SCHOOL,70,377,402,370,1149.0,EAST SIDE COMMUNITY HS,19.000000,21.000000,...,0.0,"420 East 12 Street\nNew York, NY 10009\n(40.72...",3.0,2.0,34.0,1005974.0,1.004390e+09,East Village ...,40.729783,-73.983041
3,01M509,MARTA VALLE HIGH SCHOOL,44,390,433,384,1207.0,0,129.028846,197.038462,...,0.0,"145 Stanton Street\nNew York, NY 10002\n(40.72...",3.0,1.0,3001.0,1004323.0,1.003540e+09,Chinatown ...,40.720569,-73.985673
4,01M539,"NEW EXPLORATIONS INTO SCIENCE, TECHNOLOGY AND ...",159,522,574,525,1621.0,"NEW EXPLORATIONS SCI,TECH,MATH",255.000000,377.000000,...,0.0,"111 Columbia Street\nNew York, NY 10002\n(40.7...",3.0,2.0,2201.0,1004070.0,1.003350e+09,Lower East Side ...,40.718725,-73.979426


In [76]:
combined = combined.drop(columns=['SCHOOLWIDE PUPIL-TEACHER RATIO','priority08','priority09','priority10'])

#### We now have a clean dataset we can base our analysis. Mapping the statistics out on a school district level might be an interesting way to analyze them. Adding a column to the dataset that specifies the school district helps us accomplish this.

The school district is just the first two characters of the DBN. We can apply a function over the DBN column of combined that pulls out the first two letters.

In [77]:
def extract(data):
    return data[0:2]

combined['school_dist'] = combined['DBN'].apply(extract)
combined['school_dist']

0      01
1      01
2      01
3      01
4      01
       ..
359    32
360    32
361    32
362    32
363    32
Name: school_dist, Length: 364, dtype: object

In [78]:
correlations = combined.corr()
correlations = correlations['sat_score']
print(correlations.to_string())

sat_score                               1.000000
AP Test Takers                          0.523140
Total Exams Taken                       0.514333
Number of Exams with scores 3 4 or 5    0.463245
Total Cohort                            0.325144
Total Grads - % of cohort               0.601881
Total Regents - % of cohort             0.672783
Total Regents - % of grads              0.504936
Advanced Regents - % of cohort          0.771664
Advanced Regents - % of grads           0.742616
Regents w/o Advanced - % of cohort     -0.151933
Regents w/o Advanced - % of grads      -0.447797
Local - % of cohort                    -0.482090
Local - % of grads                     -0.504936
Still Enrolled - % of cohort           -0.522865
Dropped Out - % of cohort              -0.458809
CSD                                     0.042923
NUMBER OF STUDENTS / SEATS FILLED       0.394671
NUMBER OF SECTIONS                      0.362489
AVERAGE CLASS SIZE                      0.382280
SIZE OF SMALLEST CLA

Unsurprisingly, SAT Critical Reading Avg. Score, SAT Math Avg. Score, SAT Writing Avg. Score, and sat_score are strongly correlated with sat_score.

We can also make some other observations:

- total_enrollment has a strong positive correlation with sat_score. This is surprising because we'd expect smaller schools where students receive more attention to have higher scores. However, it looks like the opposite is true - larger schools tend to do better on the SAT.
- Other columns that are proxies for enrollment correlate similarly. These include total_students, N_s, N_p, N_t, AP Test Takers, Total Exams Taken, and NUMBER OF SECTIONS.
- Both the percentage of females (female_per) and number of females (female_num) at a school correlate positively with SAT score, whereas the percentage of males (male_per) and the number of males (male_num) correlate negatively. This could indicate that women do better on the SAT than men.
- Teacher and student ratings of school safety (saf_t_11, and saf_s_11) correlate with sat_score.
- Student ratings of school academic standards (aca_s_11) correlate with sat_score, but this does not hold for ratings from teachers and parents (aca_p_11 and aca_t_11).
- There is significant racial inequality in SAT scores (white_per, asian_per, black_per, hispanic_per).
- The percentage of English language learners at the school (ell_percent, frl_percent) has a strong negative correlation with SAT scores.
- Since enrollment seems to have such a strong correlation, let's make a scatterplot of total_enrollment vs sat_score. Each point in the scatterplot represents a high school, so we'll be able to see if there are any interesting patterns.

In [79]:
px.scatter(combined, x='total_enrollment', y='sat_score')

Judging from the plot we just created, it doesn't appear there's a strong correlation between sat_score and total_enrollment.
However, there's an interesting cluster of points at the bottom left where total_enrollment and sat_score are both low. This cluster may be what's making the r value so high. It's worth extracting the names of the schools in this cluster so we can research them.

In [80]:
low_enrollment = combined[combined["total_enrollment"] < 1000]
low_enrollment = low_enrollment[low_enrollment["sat_score"] < 1000]
low_enrollment["SCHOOL NAME"]

91                  INTERNATIONAL COMMUNITY HIGH SCHOOL
126                 ACADEMY FOR LANGUAGE AND TECHNOLOGY
127                     BRONX INTERNATIONAL HIGH SCHOOL
140               KINGSBRIDGE INTERNATIONAL HIGH SCHOOL
142               INTERNATIONAL SCHOOL FOR LIBERAL ARTS
177    PAN AMERICAN INTERNATIONAL HIGH SCHOOL AT MONROE
180                       HIGH SCHOOL OF WORLD CULTURES
189                  BROOKLYN INTERNATIONAL HIGH SCHOOL
226       INTERNATIONAL HIGH SCHOOL AT PROSPECT HEIGHTS
238                          IT TAKES A VILLAGE ACADEMY
254                           MULTICULTURAL HIGH SCHOOL
287              PAN AMERICAN INTERNATIONAL HIGH SCHOOL
Name: SCHOOL NAME, dtype: object

#### Our research revealed that most of the high schools with low total enrollment and low SAT scores have high percentages of English language learners. This indicates that it's actually ell_percent that correlates strongly with sat_score, rather than total_enrollment.

In [81]:
px.scatter(combined, x='ell_percent', y='sat_score', trendline='ols')

One way to make very granular statistics easier to read is to aggregate them. In this case, we aggregate by district, which enables us to understand how ell_percent varies district-by-district instead of the unintelligibly granular school-by-school variation.

In [82]:
districts = combined.groupby('school_dist', as_index=False).agg(np.mean)
districts.head()

,school_dist,sat_score,AP Test Takers,Total Exams Taken,Number of Exams with scores 3 4 or 5,Total Cohort,Total Grads - % of cohort,Total Regents - % of cohort,Total Regents - % of grads,Advanced Regents - % of cohort,...,postcode,total_students,number_programs,Community Board,Council District,Census Tract,BIN,BBL,lat,lon
0,01,1354.500000,116.681090,173.019231,135.800000,93.500000,72.416667,66.833333,89.633333,15.783333,...,10003.166667,659.500000,1.333333,3.000000,1.500000,1106.833333,1.004144e+06,1.003302e+09,40.719022,-73.982377
1,02,1295.638184,128.908454,201.516827,157.495833,158.647849,77.383722,65.300228,83.571090,17.940832,...,10023.770833,621.395833,1.416667,4.204132,2.963052,1132.263582,1.103797e+06,1.070382e+09,40.739699,-73.991386
2,03,1293.443035,156.183494,244.522436,193.087500,183.384409,76.400071,65.174201,83.731393,16.397974,...,10023.750000,717.916667,2.000000,7.500000,6.500000,166.666667,1.034931e+06,1.012833e+09,40.781574,-73.977370
3,04,1224.142857,129.016484,183.879121,151.035714,113.857143,77.885714,58.071429,71.600000,14.114286,...,10029.857143,580.857143,1.142857,11.000000,8.000000,2637.000000,1.055874e+06,1.016681e+09,40.793449,-73.943215
4,05,1285.062687,85.722527,115.725275,142.464286,143.677419,77.197701,70.158611,89.888258,22.316732,...,10030.142857,609.857143,1.142857,9.428571,8.142857,9372.571429,1.063080e+06,1.019721e+09,40.817077,-73.949251


In [83]:
survey_fields = [
    'sat_score',
    "rr_s", 
    "rr_t", 
    "rr_p", 
    "N_s", 
    "N_t", 
    "N_p", 
    "saf_p_11",  
    "eng_p_11", 
    "aca_p_11", 
    "saf_t_11", 
    "com_t_11", 
    "eng_t_11", 
    "aca_t_11", 
    "saf_s_11", 
    "com_s_11", 
    "eng_s_11", 
    "aca_s_11", 
    "saf_tot_11", 
    "com_tot_11", 
    "eng_tot_11", 
    "aca_tot_11",
]
survey_corr = combined.loc[:,survey_fields]
sat_survey = pd.DataFrame(survey_corr.corr()['sat_score'])

In [84]:
sat_survey.reset_index(inplace=True)

In [85]:
px.bar(sat_survey, x='sat_score', y='index', orientation='h', color='sat_score')

#### We can observe a rather medium correlation of sat_score and the number of parent respondents and the number of student respondents.

### <center> Now let's take a close look at the safety score.

In [86]:
px.scatter(combined, y='saf_s_11', x='sat_score', trendline='ols')

*Looking at the distribution, we can say that
students who rated their safety score from 6.5 and higher got a higher sat_score (1400 and higher).*

### <center> Let's compute safety scores.

In [87]:
saf_by_borough = combined.groupby('borough',as_index=False).agg({'saf_tot_11':'mean'}).sort_values(by='saf_tot_11', ascending=False)
saf_by_borough

,borough,saf_tot_11
2,Manhattan,7.473333
3,Queens,7.387500
0,Bronx,7.313830
4,Staten Island,7.200000
1,Brooklyn,7.129245


*According to students, the Manhattan and Queens boroughs are the safest, and Brooklyn is the least.*

### <center> Investigation racial differences in SAT score.

In [88]:
racial_corr = combined[['sat_score','white_per','asian_per','black_per','hispanic_per']].corr()['sat_score'].reset_index()
px.bar(racial_corr, x='sat_score', y='index', orientation='h', color='sat_score')

*We can observe a strong correlation with white_per and
asian_per and a medium negative correlation with hispanic_per. \
I dare say that the negative correlation of hispanic and black races is associated with insufficient funding in the schools where students study.\
(per - percentage of students in school)*

### <center> Explore schools with low SAT scores and high values for hispanic_per.

In [89]:
hispanic_corr = combined[['sat_score','hispanic_per']]
px.scatter(hispanic_corr, x='sat_score', y='hispanic_per', trendline='ols')

In [90]:
hispanic_high = combined[combined['hispanic_per'] > 95]['SCHOOL NAME']
hispanic_high

44                         MANHATTAN BRIDGES HIGH SCHOOL
82      WASHINGTON HEIGHTS EXPEDITIONARY LEARNING SCHOOL
89     GREGORIO LUPERON HIGH SCHOOL FOR SCIENCE AND M...
126                  ACADEMY FOR LANGUAGE AND TECHNOLOGY
142                INTERNATIONAL SCHOOL FOR LIBERAL ARTS
177     PAN AMERICAN INTERNATIONAL HIGH SCHOOL AT MONROE
254                            MULTICULTURAL HIGH SCHOOL
287               PAN AMERICAN INTERNATIONAL HIGH SCHOOL
Name: SCHOOL NAME, dtype: object

*It seems that all these schools are educational institutions for Spanish-speaking immigrants with poor command of English, which may explain the low sat_score.*

In [91]:
hispanic_low = combined.query('hispanic_per < 10 & sat_score > 1800')['SCHOOL NAME']
hispanic_low

37                                STUYVESANT HIGH SCHOOL
152                         BRONX HIGH SCHOOL OF SCIENCE
188                       BROOKLYN TECHNICAL HIGH SCHOOL
328    QUEENS HIGH SCHOOL FOR THE SCIENCES AT YORK CO...
357                  STATEN ISLAND TECHNICAL HIGH SCHOOL
Name: SCHOOL NAME, dtype: object

*The listed schools teach according to the program of in-depth study of technical disciplines, so a high sat_score is not a surprise.*

### <center> Investigation gender differences in SAT score.

In [94]:
px.scatter(combined, y=combined['female_per'], x=combined['sat_score'])

In [95]:
sex_corr = combined[['sat_score','male_per','female_per']].corr()['sat_score'].reset_index()
px.bar(sex_corr, x='sat_score', y='index', orientation='h', color='sat_score')

*We see a barely existing positive correlation of sat_score and female and a slightly significant negative male.*

In [96]:
school_high_female = combined.query('female_per > 60 & sat_score > 1700')['SCHOOL NAME']
school_high_female

5                         BARD HIGH SCHOOL EARLY COLLEGE
26                         ELEANOR ROOSEVELT HIGH SCHOOL
60                                    BEACON HIGH SCHOOL
61     FIORELLO H. LAGUARDIA HIGH SCHOOL OF MUSIC & A...
303                          TOWNSEND HARRIS HIGH SCHOOL
Name: SCHOOL NAME, dtype: object

*These schools have a fairly strict competitive selection for admission.*

In [97]:
combined['ap_per'] = combined['AP Test Takers ']/combined['total_enrollment']
px.scatter(combined, x ='ap_per', y ='sat_score')

*We can observe a weak correlation between the variables.*

In [98]:
class_sat = combined[['AVERAGE CLASS SIZE','sat_score']]
px.scatter(class_sat, x='sat_score',y='AVERAGE CLASS SIZE')

*We may notice a weak correlation between sat_score and the average size of the class.*

In [101]:
best_school_by_borough = combined[combined['sat_score']>1850].groupby('borough') \
                                                             .agg({'SCHOOL NAME':'count'}) \
                                                             .rename(columns={'SCHOOL NAME':'amount'}) \
                                                             .sort_values(by='amount', ascending=False)
best_school_by_borough

,amount
borough,
Bronx,2
Manhattan,2
Queens,2
Staten Island,1


*As we can see, schools whose students scored the most points are located in Manhattan, Queens and Bronx. \
But what's interesting, according to the site https://www.yourlawyer.com/library/nyc-housing-prices-by-borough-and-neighborhood the Bronx borough has the cheapest housing, yet it has schools with a good level of education.*

In [102]:
diff_saf_score = combined[['saf_s_11','saf_p_11','saf_t_11']].mean()
diff_saf_score

saf_s_11    6.609141
saf_p_11    8.182418
saf_t_11    7.140934
dtype: float64

In [103]:
communication_score = combined[['com_s_11','com_p_11','com_t_11']].mean()
communication_score

com_s_11    6.093629
com_p_11    7.618407
com_t_11    6.523352
dtype: float64

In [104]:
academic = combined[['aca_s_11','aca_p_11','aca_t_11']].mean()
academic

aca_s_11    7.380332
aca_p_11    7.805495
aca_t_11    7.509066
dtype: float64

*As we can see, the survey observe a pattern of assessments in which students give the lowest scores, parents are high, and teachers are average.*